# Code Editor

In [ ]:
#| default_exp code_editor

## Todos
- ~~Full page editable area for writing code~~
- ~~Syntax highlighting~~
- ~~Line number side bar~~
- auto-language detection
- inline autocompletion
- ~~save files~~
- ~~handle multiple files~~
- execute code and show output

In [1]:
#| export
from fasthtml.fastapp import *

# Ace Editor (https://ace.c9.io/)
ace_editor = Script(src="https://cdnjs.cloudflare.com/ajax/libs/ace/1.35.0/ace.min.js")
# Flexbox CSS (http://flexboxgrid.com/)
gridlink = Link(rel="stylesheet", href="https://cdnjs.cloudflare.com/ajax/libs/flexboxgrid/6.3.1/flexboxgrid.min.css", type="text/css")

css = Style('''\
.sidebar {
    background-color: #f4f4f4;
    overflow-y: auto;
    padding: 10px;
    box-shadow: 2px 0 5px rgba(0,0,0,0.1);
    height: calc(100vh - 40px);
}

#editor-container {
    flex: 1;
    height: calc(100vh - 40px);
}

#editor {
    height: 100%;
    width: 100%;
}

.box-row {
    border: 1px solid #ccc;
}
''')

app, files, File = fast_app('data/files.db', hdrs=(ace_editor, gridlink, css), id=int, filename=str, content=str, pk='id')
rt = app.route

id_curr = 'current-file'
id_list = 'file-list'

In [ ]:
files.insert(File(filename='file1.txt', content='Hello, World!'))

In [2]:
#| export
js_code = """\
function renderEditor() {
    var editor = ace.edit("editor");
    editor.setTheme("ace/theme/monokai");
    editor.session.setMode("ace/mode/javascript");
}

function getFileContent() {
    var editor = ace.edit("editor");
    return editor.getValue();
}

renderEditor();
"""

example_code = """\
function foo(items) {
    var x = "All this is syntax highlighted";
    return x;
}"""

In [ ]:
#| export
def SaveFile():
    return Form(
        Input(type="text", id="filename", name="filename", placeholder="Filename", required=True),
        Button("Save", type="submit", hx_post="/save", target_id=id_list, hx_swap="beforeend", hx_vals="js:{content: getFileContent(), filename: filename.value}"),
        cls="col-xs-12"
    )
    ...
def Toolbar():
    return Div(
        Div(
            Select(
                Option("JavaScript", value="javascript"),
                Option("Python", value="python"),
                Option("HTML", value="html"),
                Option("CSS", value="css"),
                Option("Markdown", value="markdown"),
                id="language"
            ),
            Button("Run", id="run"),
            SaveFile(),
            cls="col-xs-12 toolbar"
        ),
        cls="row"
    )

In [ ]:
#| export
def FileRow(file: File):
    return Li(
        A(
            file.filename, hx_get=f'/files/{file.id}', target_id="editor-container", hx_swap="innerHTML",
            hx_on="htmx:afterSwap: renderEditor()",
          ),
        id=f'file-{file.id}'
    )

In [ ]:
#| export
def Sidebar():
    return Div(
        Div(
            Ul(*map(FileRow, files()), id=id_list), cls="sidebar"
        ),
        cls="col-xs-12 col-sm-3"
    )

In [3]:
#| export
def CodeEditor():
    toolbar = Toolbar()
    main = Div(
        Sidebar(),
        Div(
            Div(example_code, id="editor"),
            id="editor-container", cls="col-xs-12 col-sm-9", hx_on="htmx:afterSwap: renderEditor()"
        ),
        cls="row"
    )
    return Title("Code Editor",), Div(toolbar, main, cls="container-fluid"), Script(NotStr(js_code))

In [4]:
show(CodeEditor())

In [9]:
#| export
@rt("/")
def get():
    return CodeEditor()

In [ ]:
#| export
@rt("/files/{id}")
def get(id:int):
    return Div(files[id].content, id="editor", cls="ace_editor ace-tm")#, hx_on="htmx:afterSwap: renderEditor()"),

In [ ]:
#| export
@rt("/save")
def post(filename: str, content: str):
    file = File(filename=filename, content=content, id=len(files()) + 1)
    files.insert(file)
    return FileRow(file)

In [53]:
#| eval: false
#| hide
from nbdev.export import nb_export
nb_export('code_editor.ipynb', '.')